# Exploring UV extinction curves

## Learning Goals

By the end of this turorial, you will be able to:

- Understand what a UV extinction curve is and how it is used to study dust properties
- Create your own UV extinction curve
- Learn the difference in the trend for curves corresponding to the Small and Large Magellanic Clouds (LMC and SMC) 

## Introduction

**Extinction curve background**: An extinction curve represents the wavelength dependence of dust extinction. It compares the inherent Spectral Energy Distribution (SED) (~dust-free) of a star to the observed SED affected by dust extinction. Normally these curves are created by representing $k(\lambda-V)$ versus $1/\lambda$, with $\lambda$ being wavelength and $k(\lambda-V)$:

   $k(\lambda-V)$ = $\frac{E(\lambda-V)}{E(B-V)}$ = $\frac{m(\lambda-V)-m(\lambda-V)_o}{(B-V)-(B-V)_o}$ = $\frac{A(\lambda)-A(V)}{A(B)-A(V)}$

With the $x_o$ terms refering to the star that is nearly unaffected by dust. In order to perform this comparison, the stars should have the same spectral type. 

**Defining some terms**:

- **Color index**: difference between magnitude of a star in 2 different passbands, typically B and V. Symbol: $(B-V)$.
- **Extinction**: measure of interstellar reddening quantified by the difference in magnitudes. Symbol: $A_\lambda$.
- **Spectral type**: stellar classification from hotter (O stars) to cooler (M stars). Temperature defines a star's "color" and surface brightness.
- **FITS file**: Flexible Image Transport System. File often used in astronomy to store images and tables.

## Imports 

The first step will be to import the libraries we will be using throughout this tutorial: 

- _Observations_ from _astroquery.mast_ to query the Barbara A. Mikulski Archive for Space Telescopes (MAST).
- _fits_ from _astropy.io_ for accessing FITS files
- _matplotlib.pyplot_ for plotting data
- _numpy_ for array manipulations

In [ ]:
from astroquery.mast import Observations
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

**Warning**: If you have not installed the astroquery package in your computer, you may need to. Information about astroquery can be found <a href="https://astroquery.readthedocs.io/en/latest/">here</a>.

## Main content

### Loading data

The next step is to find the data file we will use. This is similar to searching through the <a href="https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html">MAST portal</a> since we will be using specific keywords to find the file. The target name of the first star we will be studying is AzV 214 (observation ID: swp22372), collected by International Ultraviolet Explorer (IUE). This will be the reddened star by dust. 

In [ ]:
IUEobs = Observations.query_criteria(obs_collection="IUE", obs_id="swp22372")
data_products = Observations.get_product_list(IUEobs)
yourProd = Observations.filter_products(data_products, extension='swp22372mxlo_vo.fits')

yourProd

Now that we have the data file, we can download it using the information contained in the table shown above by doing:

In [ ]:
Observations.download_products(yourProd, mrp_only = False, cache = False) 

If the download does not happen automatically, by clicking on the URL displayed above you will be able to download the file. In order to have a more complete image of our target, let's download a different observation for the same object, but in a different wavelength range, we can use the following observation ID: lwr17263 (this is still AzV 214).

In [ ]:
IUEobs2 = Observations.query_criteria(obs_collection="IUE", obs_id="lwr17263")
data_products2 = Observations.get_product_list(IUEobs2)
yourProd2 = Observations.filter_products(data_products2, extension='lwr17263mxlo_vo.fits')
Observations.download_products(yourProd2, mrp_only = False, cache = False) 

Now let's do the same for the unreddened star. The target name of this star is AzV 380 (observation IDs: swp10319, lwr17265) and was also collected by IUE.

In [ ]:
IUEobs3 = Observations.query_criteria(obs_collection="IUE", obs_id="swp10319")
data_products3 = Observations.get_product_list(IUEobs3)
yourProd3 = Observations.filter_products(data_products3, extension='swp10319mxlo_vo.fits')
Observations.download_products(yourProd3, mrp_only = False, cache = False) 

IUEobs4 = Observations.query_criteria(obs_collection="IUE", obs_id="lwr17265")
data_products4 = Observations.get_product_list(IUEobs4)
yourProd4 = Observations.filter_products(data_products4, extension='lwr17265mxlo_vo.fits')
Observations.download_products(yourProd4, mrp_only = False, cache = False) 

Again, if the download does not happen automatically, by clicking on the URLs displayed above, you will be able to download the necessary data to continue this tutorial.

### File information

Now, let's explore the FITS file that we got for the reddened star.

In [ ]:
filename = "./mastDownload/IUE/swp22372/swp22372mxlo_vo.fits"
fits.info(filename)

- **No. 0 (PRIMARY)**: This HDU contains meta-data related to the entire file.
- **No. 1 (Spectral Container)**: This HDU contains the spectral profile of the target as a function of wavelength.

The header of the file contains additional information about the data, such as the wavelength reference frame or the reference equinox. It can be accessed in the following way (only some part of the information contained in the header is printed in this section, the user is encouraged to print the complete header if they are interested to see the whole information contained in it).

In [ ]:
with fits.open(filename) as hdulist: 
    header1 = hdulist[1].header

print(repr(header1[54:61]))

The data contained in this fits failed can be accessed as shown below:

In [ ]:
with fits.open(filename) as hdulist:
    data0 = hdulist[1].data

wav = data0[0][0] # angstrom, A
flux = data0[0][1] # ergs/cm2/sec/A

Let's do the same for the other files and plot the logarithm of their flux versus the inverse of the wavelength, as it is normally done in this type of studies.

In [ ]:
filename2 = "./mastDownload/IUE/lwr17263/lwr17263mxlo_vo.fits"
with fits.open(filename2) as hdulist:
    data2 = hdulist[1].data

wav2 = data2[0][0] # A
flux2 = data2[0][1] # ergs/cm2/sec/A

filename3 = "./mastDownload/IUE/swp10319/swp10319mxlo_vo.fits"
with fits.open(filename3) as hdulist:
    data3 = hdulist[1].data

wav3 = data3[0][0] # A
flux3 = data3[0][1] # ergs/cm2/sec/A

filename4 = "./mastDownload/IUE/lwr17265/lwr17265mxlo_vo.fits"
with fits.open(filename4) as hdulist:
    data4 = hdulist[1].data

wav4 = data4[0][0] # A
flux4 = data4[0][1] # ergs/cm2/sec/A

wavinv = 1/(wav*1e-4) # mu-m ^-1
wavinv2 = 1/(wav2*1e-4) # mu-m ^-1
wavinv3 = 1/(wav3*1e-4) # mu-m ^-1
wavinv4 = 1/(wav4*1e-4) # mu-m ^-1

fig = plt.figure()
ax = plt.subplot(111)

plt.plot(wavinv,np.log10(flux),'r',label='swp22372')
plt.plot(wavinv2,np.log10(flux2),'r:',label='lwr17263')
plt.text(4, -12.8, 'AzV 214', fontsize = 11)
plt.plot(wavinv3,np.log10(flux3),'b',label='swp10319')
plt.plot(wavinv4,np.log10(flux4),'b:',label='lwr17265')
plt.text(7, -13.4, 'AzV 380', fontsize = 11)

ax.set_xlabel('1/$\lambda$ ($\mu m^{-1}$)')
ax.set_ylabel(r'log(Flux (ergs $cm^{-2}$ $s^{-1}$ $\AA^{-1}$))')
ax.set_xlim([3, 8])
ax.set_ylim([-14, -12.5])

plt.legend(loc='best',frameon=False)

plt.show()

**Heads-up**: when plotting the figure, you may get some warning messages, this are normal and do not mean you are doing it wrong, python may have just run into some non-logarithm friendly values, but we can still work with this data!

Let's now use <a href="http://simbad.cds.unistra.fr/simbad/">SIMBAD</a> database to look for the fluxes in the B and V bands for both stars. We can do a simple query using the identifier of both stars. The magnitudes can be found under the 8th subgroup presented below the name of the stars, called 'Fluxes', since SIMBAD can provide you with either the flux or the magnitude of the star in those bands.

- AzV 214: $m_V = 13.39$, $m_B = 13.40$.
- AzV 380: $m_V = 13.53$, $m_B = 13.43$.

From these values we can directly calculate the value of $E(B-V) = (B-V)-(B-V)_o$.

In [ ]:
V_214 = 13.39
B_214 = 13.40

V_380 = 13.53
B_380 = 13.43

E_B_V = (B_214-V_214)-(B_380-V_380)

print("The value of E(B-V) is equal to",E_B_V)

So finally, let's plot the extinction curve!

In [ ]:
plt.figure()
plt.plot(wavinv,np.abs((np.log(flux/flux3)-np.log(V_214/V_380))/E_B_V),'o',markersize=2)
plt.plot(wavinv4,np.abs((np.log(flux2/flux4)-np.log(V_214/V_380))/E_B_V),'o',markersize=2)
plt.xlim([3,8])
plt.ylim([0,12])
plt.xlabel(r'$1/\lambda$ $[\mu m^{-1}]$')
plt.ylabel(r'$E(\lambda-V)/E(B-V)$')
plt.title('AzV 214')
plt.show()

This is the typical shape encountered for extinction curves corresponding to the Small Magellanic Cloud (SMC), as can be seen in this <a href="https://arxiv.org/pdf/astro-ph/0305257.pdf">article</a>. 

In the article the extinction curve corresponding to AzV 398 can also be found, in addition to a parametrization of the extinction curve that could be applied to these data points as an extension of this project.

## Exercises

Now you can try to do it yourself! Try to obtain the extinction curve of a Large Magellanic Cloud (LMC) following the steps presented for the SMC one and look for the differences in the trend between the two.

### Load the data

The targets for this exercise will be:

- Sk -69 206 (Observation IDs: swp36552,lwp15751) (reddened star)
- Sk -67 5 (Observation IDs: swp04827,lwr04170) (unreddened star)

First, do the database search for the reddened star:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
IUEobs = Observations.query_criteria(obs_collection="IUE", obs_id="swp36552")
data_products = Observations.get_product_list(IUEobs)
yourProd = Observations.filter_products(data_products, extension="swp36552mxlo_vo.fits")

yourProd

Now, download the data as before:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
Observations.download_products(yourProd, mrp_only = False, cache = False) 

And again, in order to cover a wider wavelength range, let's also download the other part of the spectra using the second observation ID provided:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
IUEobs2 = Observations.query_criteria(obs_collection="IUE", obs_id="lwp15751")
data_products2 = Observations.get_product_list(IUEobs2)
yourProd2 = Observations.filter_products(data_products2, extension="lwp15751mxlo_vo.fits")
Observations.download_products(yourProd2, mrp_only = False, cache = False) 

Now, let's do the same for the unreddened star:

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
IUEobs3 = Observations.query_criteria(obs_collection="IUE", obs_id="swp04827")
data_products3 = Observations.get_product_list(IUEobs3)
yourProd3 = Observations.filter_products(data_products3, extension="swp04827mxlo_vo.fits")
Observations.download_products(yourProd3, mrp_only = False, cache = False) 

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
IUEobs4 = Observations.query_criteria(obs_collection="IUE", obs_id="lwr04170")
data_products4 = Observations.get_product_list(IUEobs4)
yourProd4 = Observations.filter_products(data_products4, extension="lwr04170mxlo_vo.fits")
Observations.download_products(yourProd4, mrp_only = False, cache = False) 

### Open the fits file, explore it and save the data

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
filename = "./mastDownload/IUE/swp36552/swp36552mxlo_vo.fits"
fits.info(filename)

with fits.open(filename) as hdulist:
    data1 = hdulist[1].data

wav = data1[0][0] # A
flux = data1[0][1] # ergs/cm2/sec/A

wavinv = 1/(wav*1e-4) # mu-m

filename2 = "./mastDownload/IUE/lwp15751/lwp15751mxlo_vo.fits"
fits.info(filename2)

with fits.open(filename2) as hdulist:
    data2 = hdulist[1].data

wav2 = data2[0][0] # A
flux2 = data2[0][1] # ergs/cm2/sec/A

wavinv2 = 1/(wav2*1e-4) # mu-m

filename3 = "./mastDownload/IUE/swp04827/swp04827mxlo_vo.fits"
fits.info(filename3)

with fits.open(filename3) as hdulist:
    data3 = hdulist[1].data

wav3 = data3[0][0] # A
flux3 = data3[0][1] # ergs/cm2/sec/A

wavinv3 = 1/(wav3*1e-4) # mu-m

filename4 = "./mastDownload/IUE/lwr04170/lwr04170mxlo_vo.fits"
fits.info(filename4)

with fits.open(filename4) as hdulist:
    data4 = hdulist[1].data

wav4 = data4[0][0] # A
flux4 = data4[0][1] # ergs/cm2/sec/A

wavinv4 = 1/(wav4*1e-4) # mu-m

### Plot the spectra of both stars

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
fig = plt.figure()
ax = plt.subplot(111)

plt.plot(wavinv,np.log10(flux),'r',label='swp36552')
plt.plot(wavinv2,np.log10(flux2),'r:',label='lwp15751')
plt.text(7, -14, 'Sk -69 206', fontsize = 11)
plt.plot(wavinv3,np.log10(flux3),'b',label='swp04827')
plt.plot(wavinv4,np.log10(flux4),'b:',label='lwr04170')
plt.text(7, -11.2, 'Sk -67 5', fontsize = 11)

ax.set_xlabel('1/$\lambda$ ($\mu m^{-1}$)')
ax.set_ylabel(r'log(Flux (ergs $cm^{-2}$ $s^{-1}$ $\AA^{-1}$))')
ax.set_xlim([3, 8])
ax.set_ylim([-15, -10.5])

plt.legend(loc='best',frameon=False)

plt.show()

### Find the values for the B and V bands in SIMBAD

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
V_69_206 = 12.82
B_69_206 = 12.99

V_67_5 = 11.377
B_67_5 = 11.2

E_B_V = (B_69_206-V_69_206)-(B_67_5-V_67_5)

print("The value of E(B-V) is equal to",E_B_V)

### Create the extinction curve

In [ ]:
# You can copy, paste and modify the code corresponding to this part that we used for the SMC cloud here
plt.figure()
plt.plot(wavinv,np.abs((np.log(flux/flux3)-np.log(12.12/11.0))/E_B_V),'o',markersize=2)
plt.plot(wavinv2,np.abs((np.log(flux2/flux4)-np.log(12.12/11.0))/E_B_V),'o',markersize=2)
plt.xlim([3,8])
plt.ylim([3,13])
plt.xlabel(r'$1/\lambda$ $[\mu m^{-1}]$')
plt.ylabel(r'$E(\lambda-V)/E(B-V)$')
plt.title('Sk -69 206')
plt.show()

Can you see the difference in the trend followed by the extinction curve between this case (LMC) and the one before (SMC)? More information about the different trends can be found in the following <a href="https://arxiv.org/pdf/astro-ph/0305257.pdf">article</a>.

## Additional Resources

For more information about the MAST archive and details about mission data: 

<a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
<a href="https://archive.stsci.edu/iue/">International Ultraviolet Explorer Page (MAST)</a> <br>
<a href="https://archive.stsci.edu/iue/mdr_help.html">IUE Data Retrieval Help Page</a> 

For more information about extinction curves:


<a href="https://mast.stsci.edu/api/v0/index.html">A quantitative comparison of SMC, LMC and Milky Way UV to NIR extinction curves</a> <br>

## About this Notebook

**Author**: Clara Puerto Sánchez, ScienceBetter consultant <br>
**Updated on**: 19/06/2022

## Citations

If you use `astropy`for published research, please cite the
authors. Follow these links for more information about citing `astropy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>